https://github.com/sharkwithkids/naver.git

In [16]:
import pandas as pd
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# CSV 파일에서 titleId 값을 가져오기
title_df = pd.read_csv("naver_listly_updated.csv")  # title.csv 파일을 불러오기
id_list_total = title_df['ID'].astype(str).tolist()  # titleId 칼럼의 값들을 문자열로 변환하여 리스트로 저장

num_id_list_total = len(id_list_total)  # 2468
num_id_csv_files = num_id_list_total // 100  # 24

In [17]:
def make_data(id_list):

    # URL 생성
    base_url = 'https://comic.naver.com/webtoon/list?titleId='
    urls = [f"{base_url}{num}" for num in id_list]

    # WebDriver 설정
    options = Options()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    driver = webdriver.Chrome(options=options)

    # 데이터 저장 리스트
    data = []

    # URL 순회하며 정보 수집
    for url in urls:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 제목
        title_h2 = soup.find("h2", class_="EpisodeListInfo__title--mYLjC")
        title_text = title_h2.get_text() if title_h2 else "제목을 찾을 수 없음"

        # 줄거리(스토리)
        story_p = soup.find("p", class_="EpisodeListInfo__summary--Jd1WG")
        story_text = story_p.get_text(strip=True) if story_p else "스토리를 찾을 수 없음"

        # 태그
        tags_div = soup.find("div", class_="TagGroup__tag_group--uUJza")
        if tags_div:
            tags = [tag.get_text() for tag in tags_div.find_all("a", class_="TagGroup__tag--xu0OH")]
            tags_text = ', '.join(tags)
        else:
            tags_text = "태그 그룹을 찾을 수 없음"

        # 에피소드 수 정보
        episode_info = soup.find("em", class_="ContentMetaInfo__info_item--utGrf")
        episode_info_text = episode_info.get_text(strip=True) if episode_info else "에피소드 정보를 찾을 수 없음"

        # 조회수 (좋아요 수)
        view_count = soup.find("span", class_="EpisodeListUser__count--fNEWK")
        view_count_text = view_count.get_text(strip=True) if view_count else "조회수를 찾을 수 없음"

        # 첫 공개일
        date_tag = soup.find("span", class_="date")
        date_text = date_tag.get_text(strip=True) if date_tag else "날짜를 찾을 수 없음"

        # 저장
        data.append({
            'Title': title_text,
            'Story': story_text,
            'Webtoon Info': episode_info_text,
            'Like Count': view_count_text,
            '1st Date': date_text,
            '#': tags_text
        })

        # print("현재작품 :", title_text)
        time.sleep(10)
    
    # 드라이버 종료
    driver.quit()

    return data


In [ ]:
for idx in range(num_id_csv_files+1):  # idx = 0 .. 24+1
    start, end = idx*100, min(idx*100+100, num_id_list_total)
    id_list = id_list_total[start:end]
    data = make_data(id_list)
    
    # DataFrame 생성 및 출력
    df = pd.DataFrame(data)[['Title', 'Story', 'Webtoon Info', 'Like Count', '1st Date', '#']]
    df.to_csv(f'naver_crawlling_{idx}.csv', index=False, encoding='utf-8-sig')
    print(f'naver_crawlling_{idx}.csv')

naver_crawlling_0.csv ---> was created
naver_crawlling_1.csv ---> was created
naver_crawlling_2.csv ---> was created
naver_crawlling_3.csv ---> was created
naver_crawlling_4.csv ---> was created
naver_crawlling_5.csv ---> was created
naver_crawlling_6.csv ---> was created
naver_crawlling_7.csv ---> was created
naver_crawlling_8.csv ---> was created
naver_crawlling_9.csv ---> was created
naver_crawlling_10.csv ---> was created
naver_crawlling_11.csv ---> was created
naver_crawlling_12.csv ---> was created
naver_crawlling_13.csv ---> was created
naver_crawlling_14.csv ---> was created
naver_crawlling_15.csv ---> was created
naver_crawlling_16.csv ---> was created
naver_crawlling_17.csv ---> was created
naver_crawlling_18.csv ---> was created
naver_crawlling_19.csv ---> was created
naver_crawlling_20.csv ---> was created
naver_crawlling_21.csv ---> was created
naver_crawlling_22.csv ---> was created
naver_crawlling_23.csv ---> was created
naver_crawlling_24.csv ---> was created


In [15]:
import pandas as pd

num_id_list_total = 2468
num_id_csv_files = num_id_list_total // 100  # 24
df_all = pd.DataFrame()

for idx in range(num_id_csv_files+1):  # idx = 0 .. 24+1
    filename = f'naver_crawlling_{idx}.csv'
    df = pd.read_csv(filename)
    df_all = pd.concat([df_all, df], axis=0)
    
df_all.to_csv('naver_crawlling_all.csv', index=False, encoding='utf-8-sig')
df_all.head()

,Title,Story,Webtoon Info,Like Count,1st Date,#
0,치즈인더트랩,"평범한 여대생 홍설, 그리고 어딘가 수상한 선배 유정.\n미묘한 관계의 이들이 펼쳐...",312화 완결∙12세 이용가,"329,988",18.12.21,"#로맨스, #명작, #드라마&영화 원작웹툰, #캠퍼스로맨스, #삼각관계, #완결로맨스"
1,숲속의 담,어느 순간부터 성장이 멈춘 '담'은\n사람들을 피해 숲속으로 떠난다.,132화 완결∙전체연령가,"331,689",19.12.23,"#판타지, #명작, #2019 지상최대공모전, #아포칼립스, #완결판타지"
2,고래별,1926년 일제 식민 지배 하의 조선. \n17세 소녀 수아는 군산 일대 친일파 대...,108화 완결∙12세 이용가,"326,033",19.06.20,"#로맨스, #명작, #후회물, #성장드라마, #완결로맨스"
3,귀곡의 문,"귀신소굴로 유명한 삼도천동의 흉가,\n평범한(?) 세입자들의 일상 개그",190화 완결∙12세 이용가,"213,673",18.10.16,"#드라마, #오컬트, #완결드라마"
4,마루는 강쥐,"우리 집 강아지 마루가 사람이 되었다, 그것도 5살 아이로!\n강아지 + 어린아이의...",119화 완결∙전체연령가,"671,519",22.06.13,"#개그, #일상개그힐링, #감성적인, #공감, #현대, #일상, #가벼운, #무해한..."
